**PySDS Week 03 Day 02 v.1 - Exercise - Merging and reporting on data**

# Exercise 1. Create a new codebook

For this exercise, please go through all the steps in class with respect to cleaning the data on roottweets, except do this for the replytweets table. Put this in a function that you can call. It is okay if the function is very specific to replytweets, but the more generic the better. If it could also be used to clean up roottweets from a raw SQL call, this would be ideal. 

In [2]:
# Exercise 1. 
###############################################
# Answer below here 
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime 
import re
import html
import emoji
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def createCodebookTable(df):
    ''' From the lectures - creating a codebook with basic summary stats.
    '''
    df_codebook = pd.DataFrame(index=df.columns, columns=["dtype","type","min","max","count","missing"])
    df_codebook["dtype"] = [str(df[x].dtype) for x in df.columns]
    df_codebook["type"] = [str(type(df[x][0])) for x in df.columns]
    
    df_codebook["min"] = df.min() 
    df_codebook["max"] = df.max() 
    df_codebook["count"] = df.count()
    df_codebook["missing"] = df.isnull().sum()
    return df_codebook

def getEmojiCount(text):
    ''' Get a count and list of the emoji present in a string
    '''
    result = emoji.emoji_lis(text)
    emoji_list = []
    
    for i in result:
        emoji_list.append(i["emoji"])
        
    return (len(emoji_list),emoji_list)

def cleanDataFrame(df):

    df[["tweet_id", "root_tweet_id"]] = df[["tweet_id", "root_tweet_id"]].astype(int) # Convert id columns from strings to ints
    
    mentions = re.compile("@\w*") # regex for finding mentions
    df["atmention_count"] = df["text"].map(lambda x: len(set(mentions.findall(x)))) # count of mentions
    df["atmention_list"] = df["text"].map(lambda x: list(set(mentions.findall(x)))) #list of mentions

    df['text'] = df['text'].map(lambda x: html.unescape(x)) # convert hrml to unicode characters
    df["tweet_len"] = df["text"].map(lambda x: len(x)) # get tweet length
    
    df["emoji_count"] = df["text"].map(lambda x: getEmojiCount(x)[0]) # count of emojis
    df["emoji_list"] = df["text"].map(lambda x: getEmojiCount(x)[1]) # list of emoji
    
    analyzer = SentimentIntensityAnalyzer()
    df[["neg","pos","neu","compound"]] = df['text'].apply(lambda x: pd.Series(analyzer.polarity_scores(x))) # sentiment analysis on the tweet text

    return df

filename = "PySDS_ElectionData_2015_may5-6withReplies.db"
df = pd.read_sql("select * from replytweets",sqlite3.connect(filename)) # import sql db
df = cleanDataFrame(df) # clean df
display(df.head()) # preview df
codebook = createCodebookTable(df)
display(codebook)

##############################################
# Reviewer comments below here 






,tweet_id,user_id,username,text,root_tweet_id,root_tweet_username,atmention_count,atmention_list,tweet_len,emoji_count,emoji_list,neg,pos,neu,compound
0,595657686122930176,None,george_gillett,Why you shouldn’t vote tactically in Oxford We...,595660795788275712,SallyCopley,3,"[@OxUniLabour, @Oxford_Labour, @SallyCopley]",127,0,[],0.000,1.000,0.000,0.0000
1,595577507551834112,None,BrynKewley,.@nick_clegg @EdwardDaveyMP @timfarron What th...,595577717980037122,timfarron,4,"[@timfarron, @LibDems, @EdwardDaveyMP, @nick_c...",140,0,[],0.196,0.710,0.095,-0.5514
2,595580635554676736,None,BrynKewley,.@timfarron @nick_clegg @EdwardDaveyMP @LibDem...,595577717980037122,timfarron,4,"[@timfarron, @LibDems, @EdwardDaveyMP, @nick_c...",140,0,[],0.000,1.000,0.000,0.0000
3,595583346639536128,None,BrynKewley,@timfarron @nick_clegg @EdwardDaveyMP @LibDems...,595577717980037122,timfarron,4,"[@timfarron, @LibDems, @EdwardDaveyMP, @nick_c...",137,0,[],0.100,0.698,0.201,0.3521
4,595586813122760704,None,BrynKewley,.@timfarron @nick_clegg @EdwardDaveyMP @LibDem...,595577717980037122,timfarron,4,"[@timfarron, @LibDems, @EdwardDaveyMP, @nick_c...",148,0,[],0.000,1.000,0.000,0.0000


,dtype,type,min,max,count,missing
tweet_id,int64,<class 'numpy.int64'>,467161825411211264,606600684738383872,36259,0
user_id,object,<class 'NoneType'>,None,None,0,36259
username,object,<class 'str'>,001stcole,zygote23,36259,0
text,object,<class 'str'>,"""25 reasons to Vote @UKLabour "" by @grahamemor...",😎✌🏻️👍🏻 #GE2015 @TheGreenParty @TauntonGreens p...,36259,0
root_tweet_id,int64,<class 'numpy.int64'>,595377394803974144,596087022734987264,36259,0
root_tweet_username,object,<class 'str'>,19rbw17,yrossr,36259,0
atmention_count,int64,<class 'numpy.int64'>,0,11,36259,0
atmention_list,object,<class 'list'>,[],"[@zoomer56snp, @lisacameronsnp, @mgtferrier, @...",36259,0
tweet_len,int64,<class 'numpy.int64'>,8,176,36259,0
emoji_count,int64,<class 'numpy.int64'>,0,18,36259,0


# Exercise 2. Finding the happy tweets. 

Using the indepedent samples ttest function, split the data into at least two groups (e.g., by length of tweets / has emoji / has @mention, etc...). Compare two of theese groups using an independent samples t-test. (See example below). Try to find a split that will lead to a significant difference between the two splits. After trying three different splits, if there is no significant difference, simply move on. Report all three splits. If you get a significant difference on the first split, great! This can be done with either the roottweets table or the replytweets table.

In [3]:
###############################
# Example ttest code 
from scipy import stats

# r1 = [1,3,5,7,9,11]
# r2 = [1,3,4,6,8,3,6,7]
# r3 = [80,10,20,31,4,45]
# print(stats.ttest_ind(r1,r2)) # for paired samples it's ttest_rel(x,y)
# print(stats.ttest_ind(r1,r3).pvalue)
# statistics > 2, significant

################################
# Answer below here 

def SplitDataTest(feature, value): # create function that splits data on some feature and value
    group1 = df["compound"][df[feature] <= value]
    group2 = df["compound"][df[feature] > value]  
    return stats.ttest_ind(group1,group2)

print("Mentions:", SplitDataTest("atmention_count", 0)) # tweets with mentions vs without

print("Emojis:", SplitDataTest("emoji_count", 0)) # tweets woth emojis vs without

print("Tweet length >70 characters:", SplitDataTest("tweet_len", 70)) # long vs short tweets


################################
# Peer review comments below here 






Mentions: Ttest_indResult(statistic=0.752386241403458, pvalue=0.4518237026506444)
Emojis: Ttest_indResult(statistic=-11.749716985156695, pvalue=8.093336463183501e-32)
Tweet length >70 characters: Ttest_indResult(statistic=6.38485107003794, pvalue=1.7363668469672953e-10)


# Exercise 3. Finding the tweetstorm tweet. 

We want to find out what tweet inspired the most negative replies. First, create a 'grouped_reply_tweets' table/DataFrame.  It should have the roottweet_id, the count of replies, and the average sentiment score for pos, neg, neu. 

Filter this table to those roottweets that have > 1 replies. Look for the tweet(s) with the maximum average negative sentiment. If there are more than one with the same max negative sentiment, take the roottweet(s) with the most replies. Use these tweet IDs to look up the tweet(s) in the roottweets table. What tweet was it that prompted such negativity? Report your output as follows: 

```
The maximum negative sentiment score was %s. The replies that got this score were:

Tweet 1.
<tweet> 

Tweet 2. 
<tweet>

etc...

The root tweet that inspired such negativity was written by @<user>. It was: 


```

In [4]:
################################
# Answer below here 

mean_df = df.groupby("root_tweet_id").mean()
mean_df['count'] = df["root_tweet_id"].value_counts()

mean_df = mean_df[mean_df['count']>1]

neg_score = mean_df['neg'].max()
neg_id = mean_df['neg'].idxmax()
print(neg_id)
replies = df['text'][df['root_tweet_id']==neg_id]

roottweets_df = pd.read_sql("select * from roottweets",sqlite3.connect(filename))
roottweets_df['tweet_id']=roottweets_df['tweet_id'].astype(np.int64)
roottweets_df = roottweets_df.set_index('tweet_id', drop=True)

print("The maximum negative sentiment score was %s. The replies that got this score were: " %neg_score)

for n, i in enumerate(replies):
    print('Tweet %d.\n%s' %(n+1,i))

print()
print("The root tweet that inspired such negativity was written by @%s" %roottweets_df.loc[neg_id, 'username'])
print("It was: \n%s" %roottweets_df.loc[neg_id, 'text'])

################################
# Peer review comments below here 



595663754622304256
The maximum negative sentiment score was 0.455. The replies that got this score were: 
Tweet 1.
.@IainMcGill No need Iain, no need. Honestly.
Tweet 2.
@alexmassie @IainMcGill A jambo tory in Leith - poor bastard.

The root tweet that inspired such negativity was written by @IainMcGill
It was: 
@alexmassie I'll can canvass you in half an hours time if you need it evened out...
